# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [13]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""
# sql 문 마지막에 `;` 은 붙이지 않는다.

In [14]:
import pymysql

try:
    conn = None
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user="scott",        # username: str
        password="tiger",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer") # 1
    cursor.execute(create_sql) 
    
finally:
    # 4. 연결닫기(끊기)
    if conn: # conn != None
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [5]:
from datetime import date, datetime, time
a = date(2000, 10, 20)                                    # 날짜 객체 생성 - 년, 월, 일
print(a.year, a.month, a.day)                             # data객체에서 년/월/일 조회
b = datetime(2000,  10, 20, 12, 23, 50)                       # 날짜시간 객체 생성 - 년, 월, 일, 시, 분, 초 지정. (초는 생략가능)
print(b.year, b.month, b.day, b.hour, b.minute, b.second) # 년/월/일/시/분/초 조회

print(datetime.now())  # 실행시점의 일시로 datetime객체 생성 (sql: now())
print(date.today())    # 실행시점의 날짜로 date객체 생성    (sql: curdate())

2000 10 20
2000 10 20 12 23 50
2024-10-10 10:43:29.470829
2024-10-10


## DML

### insert


In [15]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [17]:
# connection, cursor 생성 -> with 문으로 작성.
# manual commit 이 default. 
##  DML(insert/update/delete) 처리 후 commit을 실행해야한다.
with pymysql.connect(host="127.0.0.1", port=3306, user="scott", password="tiger", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) # 처리행수(insert/delete/update된행수, select 조회행수)
        print("처리 행수:", result)
        # commit 처리 
        conn.commit()
   

IntegrityError: (1062, "Duplicate entry 'lee1@naver.com' for key 'customer.email'")

### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [19]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")

# SQL 실행
insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, 175.23, "2010-01-01"])
        conn.commit()
        print("처리 행수:", result)

이름: 강감찬
이메일주소: k@a.com


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [22]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [23]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)  # executemany() 결과 행수 반환.
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [24]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  171.23
변경할 고객 ID:  1


처리 행수:  1


In [25]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 Email주소:  new@email.com
변경할 키:  230
변경할 고객 ID:  1


처리 행수:  1


In [26]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
        # result = cursor.execute(delete_sql, (tall,))
        print("처리 행수: ", result)
        conn.commit()

삭제기준 키: 180


처리 행수:  3


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [27]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [29]:
resultset
# tuple(개별행 - tuple(컬럼값들 ))

((4, '유관순', 175.23),
 (5, '강감찬', 175.23),
 (6, '김인영', 165.0),
 (7, '오수철', 175.0),
 (9, '김명수', 177.0),
 (10, '이지영', 163.0))

In [31]:
# 첫번째 사람의 이름
resultset[0][1]

'유관순'

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [32]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [33]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}]

[{'id': 4, 'name': '유관순', 'tall': 175.23},
 {'id': 5, 'name': '강감찬', 'tall': 175.23},
 {'id': 6, 'name': '김인영', 'tall': 165.0},
 {'id': 7, 'name': '오수철', 'tall': 175.0},
 {'id': 9, 'name': '김명수', 'tall': 177.0},
 {'id': 10, 'name': '이지영', 'tall': 163.0}]

### fetchone()


In [44]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [7])  #(2, )
        print("조회행수:", result)
        resultset = cursor.fetchone()
        # resultset = cursor.fetchall()

조회행수: 1


In [45]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
    # print("이름:", resultset[1])
else:
    print("조회결과가 없음.")

(7, '오수철', 'def2@a.com', 175.0, datetime.date(1995, 12, 20), datetime.datetime(2024, 10, 10, 11, 23, 44))


### fetchmany()


In [48]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)

조회행수: 6


In [49]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)

((4, '유관순', datetime.date(2010, 1, 1)), (5, '강감찬', datetime.date(2010, 1, 1)))
((6, '김인영', datetime.date(2005, 1, 12)), (7, '오수철', datetime.date(1995, 12, 20)), (9, '김명수', datetime.date(2000, 2, 12)))
((10, '이지영', datetime.date(1995, 4, 21)),)
()


### select문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [51]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor:
            print(id, name, email, tall, birthday, created_at, sep=" , ")
        # for v in cursor:
        #     print(v)

조회행수: 6
4 , 유관순 , a@a.com , 175.23 , 2010-01-01 , 2024-10-10 11:17:34
5 , 강감찬 , k@a.com , 175.23 , 2010-01-01 , 2024-10-10 11:17:50
6 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2024-10-10 11:23:44
7 , 오수철 , def2@a.com , 175.0 , 1995-12-20 , 2024-10-10 11:23:44
9 , 김명수 , jkl@abc.com , 177.0 , 2000-02-12 , 2024-10-10 11:23:44
10 , 이지영 , mno@abc.com , 163.0 , 1995-04-21 , 2024-10-10 11:23:44


In [52]:
# database.py 모듈을 이용해서 hr_join DB와 관련된 select 실행.
from database import HRDao  # Dao: Data Access Object
import configparser as parser # config.ini 의 정보를 조회.

In [53]:
# configparser에서 MYSQL연결 정보 조회
props = parser.ConfigParser()  # Parser객체 생성
props.read("./config.ini")     # ini 파일 읽기
mysql_config = props['MYSQL']  # MYSQL Section "[MYSQL]"의 설정정보 조회
print("host:", mysql_config['host'])

host: 127.0.0.1


In [57]:
dao = HRDao(
    mysql_config['host'],
    3306, 
    mysql_config['user'],
    mysql_config['password'],
    mysql_config['db']
)

### 전직원 정보를 조회 - select_emp_all()
all_emp = dao.select_all_emp()
len(all_emp)

107

In [58]:
all_dept = dao.select_dept()
print(all_dept)

((110, 'Accounting', 'Seattle'), (10, 'Administration', 'Seattle'), (160, 'Benefits', 'Seattle'), (180, 'Construction', 'Seattle'), (190, 'Contracting', 'Seattle'), (140, 'Control And Credit', 'Seattle'), (130, 'Corporate Tax', 'Seattle'), (90, 'Executive', 'Seattle'), (100, 'Finance', 'Seattle'), (240, 'Government Sales', 'Seattle'), (40, 'Human Resources', 'New York'), (60, 'IT', 'San Francisco'), (230, 'IT Helpdesk', 'Seattle'), (210, 'IT Support', 'Seattle'), (170, 'Manufacturing', 'Seattle'), (20, 'Marketing', 'New York'), (220, 'NOC', 'Seattle'), (200, 'Operations', 'Seattle'), (270, 'Payroll', 'Seattle'), (70, 'Public Relations', 'New York'), (30, 'Purchasing', 'Seattle'), (260, 'Recruiting', 'Seattle'), (250, 'Retail Sales', 'Seattle'), (80, 'Sales', 'New York'), (150, 'Shareholder Services', 'Seattle'), (50, 'Shipping', 'San Francisco'), (120, 'Treasury', 'Seattle'))


In [60]:
result = dao.select_emp_by_dept(30)
result

[{'emp_id': 114,
  'emp_name': 'Den',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Manager',
  'hire_date': datetime.date(2002, 12, 7),
  'salary': '$11,000'},
 {'emp_id': 115,
  'emp_name': 'Alexander',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Manager',
  'hire_date': datetime.date(2003, 5, 18),
  'salary': '$9,100'},
 {'emp_id': 116,
  'emp_name': 'Shelli',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Clerk',
  'hire_date': datetime.date(2005, 12, 24),
  'salary': '$2,900'},
 {'emp_id': 117,
  'emp_name': 'Sigal',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Clerk',
  'hire_date': datetime.date(2005, 7, 24),
  'salary': '$2,800'},
 {'emp_id': 118,
  'emp_name': 'Guy',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Clerk',
  'hire_date': datetime.date(2006, 11, 15),
  'salary': '$2,600'},
 {'emp_id': 119,
  'emp_name': 'Karen',
  'dept_name': 'Purchasing',
  'job_title': 'Purchasing Clerk',
  'hire_date': datetime.date(2007, 8, 